In [ ]:
import pandas as pd

# Carregar os Dados
Dados_Banco = pd.read_csv('BankChurners.csv')

# Exploração Básica
print(Dados_Banco.describe())
print(Dados_Banco.info())
print(Dados_Banco.head())

# Limpeza de Dados
Dados_Banco.drop(columns=['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'], inplace=True)


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Seleção de features
features = ['Customer_Age', 'Gender', 'Education_Level', 'Months_on_book']
X = Dados_Banco[features]

# Normalização dos dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# codificação de variaveis categoricas
encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(X[['Gender', 'Education_Level']])


In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Escolha do numero de Clusters
# Metodo do cotovelo
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(X_scaled)
    wcss.append(kmeans.inertia_)
plt.plot(range(1, 11), wcss)
plt.title('Método do Cotovelo')
plt.xlabel('Número de Clusters')
plt.ylabel('WCSS')
plt.show()

# Cluster
kmeans = KMeans(n_clusters=3)
kmeans.fit(X_scaled)
clusters = kmeans.predict(X_scaled)

# Analise dos Clusters
cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)
print(cluster_centers)

In [ ]:
from sklearn.metrics import silhouette_score

# Avaliação de clusters
silhouette_avg = silhouette_score(X_scaled, clusters)
print("Silueta media para o numero de clusters =", silhouette_avg)


In [ ]:
import seaborn as sns

# Distribuição de variaveis
plt.figure(figsize=(10, 6))
sns.histplot(data=Dados_Banco, x='Customer_Age', hue=clusters, kde=True)
plt.title('Distribuição de Idade por Cluster')
plt.xlabel('Idade')
plt.ylabel('Contagem')
plt.legend(title='Cluster')
plt.show()

# Correlação entre variaveis
correlation_matrix = Dados_Banco.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Mapa de Calor da Correlação entre Variáveis')
plt.show()

# Visualizações expecificas dos clusters

# Boxplot por cluster
for feature in ['Customer_Age', 'Months_on_book']:
    plt.figure(figsize=(8, 6))
    sns.boxplot(x=clusters, y=feature, data=Dados_Banco)
    plt.title(f'Boxplot de {feature} por Cluster')
    plt.xlabel('Cluster')
    plt.ylabel(feature)
    plt.show()

# Perfis dos Clustes com grafics de radar
cluster_centers_df = pd.DataFrame(cluster_centers, columns=['Customer_Age', 'Months_on_book'])
cluster_centers_df['Cluster'] = range(0, len(cluster_centers_df))
cluster_melted = pd.melt(cluster_centers_df, id_vars='Cluster', var_name='Feature', value_name='Value')

plt.figure(figsize=(10, 8))
sns.lineplot(x='Feature', y='Value', hue='Cluster', data=cluster_melted, marker='o')
plt.title('Perfil dos Clusters com Gráficos de Radar')
plt.xlabel('Feature')
plt.ylabel('Valor')
plt.xticks(rotation=45)
plt.legend(title='Cluster')
plt.show()

# Comparação de clusters com graficos de barra
for feature in ['Gender', 'Education_Level']:
    plt.figure(figsize=(8, 6))
    sns.countplot(x=feature, hue=clusters, data=Dados_Banco)
    plt.title(f'Gráfico de Barras de {feature} por Cluster')
    plt.xlabel(feature)
    plt.ylabel('Contagem')
    plt.legend(title='Cluster')
    plt.show()
